# Hex Color to Color Name

In [ ]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

import warnings   # 경고 안뜨게
warnings.filterwarnings(action='ignore')

데이터 가져오기

In [ ]:
import pymysql
from sqlalchemy import create_engine

engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8')
engine_conn = engine.connect()

data = pd.read_sql("""
    SELECT * FROM color_config_new;
    """, engine_conn)

engine_conn.close()
print('- - - - - 완료 - - - - -')

### Change 'hex Color' to 'color name'

In [ ]:
def get_color_name(idx):
    # get request
    hex_color = data['hex'][idx][1:]
    url = f'https://www.htmlcsscolor.com/hex/{hex_color}'
    rq = requests.get(url)

    # html 가져오기
    html = rq.text

    # html 파싱하기
    bs = BeautifulSoup(html, 'html.parser')

    # 색상명 가져오기
    return bs.find('small').text

In [ ]:
from multiprocessing import Pool 

if __name__=='__main__':
    pool = Pool(processes=12)
    result = pool.map(get_color_name, range(0, len(data)))

data['color_name'] = result

In [ ]:
data

In [ ]:
engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8')
engine_conn = engine.connect()
data.to_sql(table_name, engine_conn, if_exists='replace', index=None)
engine_conn.close()
engine.dispose()